# Redis

On this page I'm getting to know reddis. It's a inmemory no-sql database.

## Using command line

Just use the `redis-cli` command to go into its shell. There is a `ping` command that answers `PONG` to check if everything is OK.

The following cell shows how this can look like.

In [1]:
%%bash
redis-cli
ping

PONG


It is also sometimes useful to be able to run `redis` commands without having to enter the redis shell interactively.

In [2]:
!redis-cli ping

PONG


## String

Is atomic thing in `redis`. There are such commands to operate on it.

|Command|What it does|
|-------|------------|
| `SET` | Sets value for the variable|
| `GET` | Get value from the key value|
| `DEL` | Delet value under key|

In [3]:
%%bash
redis-cli

SET hello world
GET hello

OK
world


After activating `redis-cli` we add a new key/value combination - it returns `OK` and then try to access the value under the `hello` key - it returns `world`, which is the value corresponding to the `hello` key.

Ok now lets try to delete just created key/value pair from redis.

In [4]:
%%bash
redis-cli

DEL hello
DEL hello
GET hello

1
0



The first execution of `del` returns `1` because the key `hello` exists, but the second `del` returns `0` because the key `hello` has already been deleted. And as proof that the key has been deleted, we try to access it with `get` and get an empty response.

## Lists

Redis has a `list` data structure. There are some operations you can perform on redis lists.

|Command|What it does|
|-------|------------|
|`RPUHS`|Pushes the value onto the right end of the list|
|`LRANGE`|Fetches a range of values from the list|
|`LINDEX`|Fetches an item at a given position in the list|
|`LPOP`|Pops the value from the left end of the list and returns it|

Now let's play around with it.

Here some values are pushed to the `list-key`.

In [5]:
%%bash
redis-cli
RPUSH list-key my name Fedor

3


By using `list-key` we can get values that were stored in the list.

In [6]:
%%bash
redis-cli
LRANGE list-key 0 -1

my
name
Fedor


By using `lindex` we can access any element you want.

In [7]:
%%bash
redis-cli

LINDEX list-key 0
LINDEX list-key 2
LINDEX list-key 1

my
Fedor
name


Now we can `rpop` items from the list. Each pop returns the last value of the list and clears it.

In [8]:
%%bash
redis-cli

RPOP list-key
RPOP list-key
RPOP list-key

Fedor
name
my


## Sets

Redis has a `set` data structure - it's a bunch of unique strings. There are some commands to use them with redis sets:

|Command|What it does|
|-------|------------|
|`SADD`|Adds an item to the set|
|`SMEMBERS`|Returns entire set of items|
|`SISMEMBER`|Checks is an item is in set|
|`SREM`|Remove the item from the set|

Next cells represents some experiments with this set of commands.

Now let's add some new values to the set. Here we add `"this value"` and `"new value"`, but `"this value"` has been added twice.

In [9]:
%%bash
redis-cli

SADD set-key "this value"
SADD set-key "new value"
SADD set-key "this value"

1
1
0


It's interesting that outputs looks like it returns `1` if the operation was successful and `0` if not. In our case the last `sadd` tried to add an already existing element.

Now, using `smembers`, we can access the set we have just created.

In [10]:
%%bash 
redis-cli
SMEMBERS set-key

this value
new value


Some practice for `sismember` and `srem` in one cell.

In [11]:
%%bash
redis-cli

SISMEMBER set-key "new value"
SREM set-key "new value"
SISMEMBER set-key "new value"

1
1
0


Before removing `sismember` it returns `1`, after removing it returns `0`. The middle `1` is the output of the `srem` command - it signals that everything has been successfully completed.

Don't forget to delete the set you just created, to ensure the same execution for this example and to exclude useless garbage in our reddis instance.

In [12]:
!redis-cli DEL set-key

(integer) 1


## Hashes

`Hash` is a key/value structure in redis. You can think of it as a small sub-redis.

You can operate with hashes using following commands.

|Command|What it does|
|-------|------------|
|`HSET`|Stores the value at the key in the hash|
|`HGET`|Fetches value at the given hash key|
|`HGETALL`|Fetches the entire hash|
|`HDEL`|Removes the key from the hash|

Here is an example of making hash with different animals and the sounds they make.

In [13]:
%%bash
redis-cli

HSET hash-key cat meow
HSET hash-key dog woh
HSET hash-key snake ssss
HSET hash-key cat mow
HSET hash-key bird chirp
HSET hash-key dog bark
HSET hash-key snake hiss
HSET hash-key cat purr
HSET hash-key bird tweet

1
1
1
0
1
0
0
0
0


Each time a new record was added to the has, 0 or 1 was printed. 1 is for cases where a new key was created, but 0 is for cases where such a case already exists.

Now let's try `hgetall`.

In [14]:
%%bash
redis-cli
HGETALL hash-key

cat
purr
dog
bark
snake
hiss
bird
tweet


Well, result may be a bit uncomfortable to read but it has structure like:

```
<key1>
<value1>
<key2>
<value2>
...
<keyN>
<valueN>
```

Don't forget to delete the hash you just created, to ensure the same execution for this example and to exclude useless garbage in our reddis instance.

In [15]:
!redis-cli DEL hash-key

(integer) 1


## Zsets

Is a data structure where the number (score) corresponds to the value. This allows you to select values by the range of their scores.

To show it's functionality we'll need following operations:

|Command|What it does|
|-------|------------|
|`ZADD`|Adds member with get given score to zset|
|`ZRANGE`|Fetches the items in the zset from their position in sorted order|

This section only describes basic concepts, find out more on the [specific page](redis/zset.ipynb).

Ok let's have experiments with this structure. Here we have zset that ranges aminal acording to their weights. For this we can use syntax `zadd <zset-name> <score1> <value1> <score2> <value2> ... <scoreN> <valueN>`

In [16]:
%%bash
redis-cli

ZADD animal-weights 50 elephant
ZADD animal-weights 20 lion
ZADD animal-weights 30 giraffe
ZADD animal-weights 10 cheetah
ZADD animal-weights 50 rhinoceros
ZADD animal-weights 30 giraffe
ZADD animal-weights 25 lion
ZADD animal-weights 20 tiger

1
1
1
1
1
0
0
1


It's output is 1 every time I add a new value, but scores can be repeated as often as you like.

Now let us try to apply `zrange` to the zset under consideration:

In [17]:
%%bash
redis-cli
ZRANGE animal-weights 0 -1

cheetah
tiger
lion
giraffe
elephant
rhinoceros


We have all the values that we tried to add once. In order of increasing scores. **Note** that `rhinoceros` have the same score as `elephant` but it diaplyed later because it was added later.

Don't forget to delete the zset you just created, to ensure the same execution for this example and to exclude useless garbage in our reddis instance.

In [18]:
!redis-cli DEL animal-weights

(integer) 1
